initial imports

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from itertools import groupby
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
import time

suffix for saving files 

In [ ]:
def output_name(in_name, suffix):
    prefix = in_name.split('.csv')[0]
    datetime = time.strftime("%Y-%m-%d_%H-%M-%S")
    return prefix + '_' + suffix + '_' + datetime + '.csv'

define fields

In [ ]:
_PART       = 'RECORDING_SESSION_LABEL'
PART        = 'Participant'
_TRIAL      = 'TRIAL_INDEX'
TRIAL       = 'Trial'
_TARGET     = 'AVERAGE_IA_1_SAMPLE_COUNT_%'
TARGET      = 'Target'
_FILL1      = 'AVERAGE_IA_3_SAMPLE_COUNT_%'
FILL1       = 'Filler1'
_COMP       = 'AVERAGE_IA_2_SAMPLE_COUNT_%'
COMP        = 'Competitor'
_FILL2      = 'AVERAGE_IA_4_SAMPLE_COUNT_%'
FILL2       = 'Filler2'
_TIME       = 'BIN_START_TIME'
TIME        = 'Time'
TYPE        = 'type'
VERSION     = 'version'
TARGET_WORD = 'target_word'


define calculated fields

In [ ]:
# calculated special
FILLAvg    = 'Filler Average'
NTAvg      = 'Non Target Average'
TCAvg      = 'Target Competitor Average'
TmC        = 'Target minus Competitor'
TmF        = 'Target minus Filler'
TmNT       = 'Target minus Non Target'
TCmF       = 'Target Competitor minus Filler'

define columns to keep when clearing up some elements

In [ ]:
KEEP_COLUMNS = [TIME, PART, TRIAL, TYPE, VERSION, TARGET, COMP, FILL1, FILL2, TARGET_WORD,
                FILLAvg, NTAvg, TCAvg, TmC, TmF, TmNT, TCmF]

define some colors and names for plotting

In [ ]:

TARGET_COLOR  = 'red'
COMP_COLOR    = 'blue'
NTargetCOLOR  = 'green'
color_dict={COMP: COMP_COLOR, TARGET: TARGET_COLOR, NTAvg: NTargetCOLOR, FILLAvg: NTargetCOLOR, TCAvg: TARGET_COLOR}


set files names

In [ ]:
F_OLD_DATA = 'data/240621_old_time_course_binning.xls.csv'
F_OLD_TOUCH_DATA ='data/240621_old_message_report.xls.csv'
F_YOUNG_DATA = 'data/240621_young_time_course_binning.xls.csv'
F_YOUNG_TOUCH_DATA ='data/240621_young_message_report.xls.csv'

load files

In [ ]:
#old_data = pd.read_csv('data/240621_old_time_course_binning.xls.csv')
#old_touch_data = pd.read_csv('data/240621_old_message_report.xls.csv')
#young_data = pd.read_csv('data/240621_young_time_course_binning.xls.csv')
#young_touch_data = pd.read_csv('data/240621_young_message_report.xls.csv')

old_data = pd.read_csv(F_OLD_DATA)
old_touch_data = pd.read_csv(F_OLD_TOUCH_DATA)
young_data = pd.read_csv(F_YOUNG_DATA)
young_touch_data = pd.read_csv(F_YOUNG_TOUCH_DATA)


just dumps the columns and sync columns to keep

In [ ]:
LOAD='load'
DIGITS='DigitsCorrect'
RESPONSE='Response'

# KEEP_COLUMNS2 = [TIME, PART, TRIAL, TYPE, LOAD, DIGITS, TARGET, COMP, FILL1, FILL2, 'Response']
old_touch_data.columns, old_data.columns

In [ ]:
#young_touch_data.loc[young_touch_data.CURRENT_MSG_TEXT.str.contains("TRIALID ")]
#young_touch_data.loc[(young_touch_data.TRIAL_INDEX == 5)]

rename columns names

In [ ]:
for data in [old_data, old_touch_data, young_data, young_touch_data]:
    data.rename(columns={
        _PART:   PART,
        _TRIAL:  TRIAL,
        _TARGET: TARGET,
        _COMP:   COMP,
        _FILL1:  FILL1,
        _FILL2:  FILL2,
        _TIME:   TIME
    }, inplace=True)

In [ ]:
# test plot
old_data[old_data[TYPE] == 'b'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
pp.show()

fix the trial column to be the numbers from 'TRIAL_LABEL' column data 

In [ ]:
old_data_1 = old_data.assign(**{TRIAL: lambda x: pd.to_numeric(x['TRIAL_LABEL'].str.strip('Trial: '))})
young_data_1 = young_data.assign(**{TRIAL: lambda x: pd.to_numeric(x['TRIAL_LABEL'].str.strip('Trial: '))})

#### get message report offset

 
 read the message report and extract TOUCH_TARGET time stamp and stimuli timestamp,
 
 calculate ofset for TOUCH_TARGET from mimimum of stimuli (when multiple stimuli exists)

In [ ]:
od1 = old_touch_data
yd1 = young_touch_data
# od1['CURRENT_MSG_TEXT']
# TOUCH_TARGET ==> Selection_Target
# Stiimuli ==> Sound_Start
    # filter messages which are not TOUCH_TARGET nor stimuli ...
od2 = od1.loc[(od1['CURRENT_MSG_TEXT'] == 'Selection_Target') |
              (od1['CURRENT_MSG_TEXT'].str.contains('Sound_Start'))]

yd2 = yd1.loc[(yd1['CURRENT_MSG_TEXT'] == 'Selection_Target') |
              (yd1['CURRENT_MSG_TEXT'].str.contains('Sound_Start'))]


#od2

#od2[od2['CURRENT_MSG_TEXT'].str.contains('Sound_Start')].pivot_table(
#    index=[PART, TRIAL], values='CURRENT_MSG_TIME', aggfunc=min
#)

#od2


In [ ]:
# show columns
od2.columns

find which trials are missing entries of stimuli touch report bundle

In [ ]:

od3 = od2.pivot_table(index=[PART], values=[
     'CURRENT_MSG_TEXT'], columns=[TRIAL], aggfunc=[len])

yd3 = yd2.pivot_table(index=[PART], values=[
     'CURRENT_MSG_TEXT'], columns=[TRIAL], aggfunc=[len])


In [ ]:
#od3, yd3

In [ ]:

    # pivot table to have columns for TOUCH_TARGET and Stimuli, have min value of CURRENT_MSG_TIME as the new value
od5 = od2.pivot_table(index=[PART, TRIAL],
                            columns=['CURRENT_MSG_TEXT'],
                            values='CURRENT_MSG_TIME', aggfunc=min)

yd5 = yd2.pivot_table(index=[PART, TRIAL],
                            columns=['CURRENT_MSG_TEXT'],
                            values='CURRENT_MSG_TIME', aggfunc=min)


#od5
    # find out missing TOUCH_TARGET trials
#od5[od5['Selection_Target'].isnull()]

In [ ]:
    # get the int value instead of float
od5_1 = od5[~od5['Selection_Target'].isnull()].astype(int)

yd5_1 = yd5[~yd5['Selection_Target'].isnull()].astype(int)
#od5_1

In [ ]:
   # fill Nan as 0, set type to int and calculate offset from Stimuli to TOUCH_TARGET
od6 = od5.fillna(0).astype(int).assign(
        TOUCH_FIXED=lambda x: x['Selection_Target'] - x['Sound_Start'])

yd6 = yd5.fillna(0).astype(int).assign(
        TOUCH_FIXED=lambda x: x['Selection_Target'] - x['Sound_Start'])


#od6


In [ ]:
   # missing TOUCH_TARGET message report
od6[od6['TOUCH_FIXED'] < 0]
yd6[yd6['TOUCH_FIXED'] < 0]

In [ ]:
    # rename only the indexs 
od7 = od6.rename_axis([PART, TRIAL])
yd7 = yd6.rename_axis([PART, TRIAL])
od7
yd7

get message report offset - finished

In [ ]:

old_touch_data_1 = od6
young_touch_data_1 = yd6

merge the data with touch data

In [ ]:
old_data_with_touch = pd.merge(old_data_1.set_index([PART, TRIAL]),
                               old_touch_data_1,
                               on=[PART, TRIAL])

young_data_with_touch = pd.merge(young_data_1.set_index([PART, TRIAL]),
                               young_touch_data_1,
                               on=[PART, TRIAL])

In [ ]:
def plot_part_trial(data2, _p, _t):
#    (_p, _t) = (411, 9)
    p = data2.loc[_p].loc[_t].plot(x=TIME, y=TARGET)
    p2 = p.bar(x=data2.loc[_p].loc[_t].TOUCH_FIXED.values[0],
               height=10, width=20, color="red")
    p3 = p.bar(x=1500, height=10, width=20, color='magenta')
    p4 = p.bar(x=2700, height=10, width=20, color='black')
    p.set_ylim(-10, 100)
    p.set_title("Part: {}, Trial: {}, Type: {}".format(
        _p, _t, data2.loc[_p].loc[_t].index[0]), fontsize=TITLE_FONT_SIZE)
    p.legend(['Target', 'cutoff {}'.format(
        data2.loc[_p].loc[_t].TOUCH_FIXED.values[0]), 'qend: 1500', 'target: 2700'], loc='upper left',
        fontsize=LEGEND_FONT_SIZE)

In [ ]:
#old_data_with_touch
#z=old_data_with_touch.set_index(TYPE)

#p = old_data_with_touch.loc[201].loc[5].plot(x=TIME, y=TARGET)

#plot_part_trial(z, 201, 5)

if we want to plot target for specific participant and trial

In [ ]:
# not working ATM
#plot_part_trial(old_data_with_touch.set_index(PART, TRIAL, TYPE), 205, 5)

In [ ]:
odt1 = old_data_with_touch
ydt1 = young_data_with_touch
odt1
ydt1

plot for reference before padding and chopping

In [ ]:
# old data
#odt1[odt1[TYPE] == 'a'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
#odt1[odt1[TYPE] == 'b'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
#odt1[odt1[TYPE] == 'c'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
#odt1[odt1[TYPE] == 'p'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()

for t in odt1[TYPE].unique():
    odt1[odt1[TYPE] == t].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
    pp.title('Old Type ' + t)
    pp.ylabel('Target Probability')
    pp.xlabel('Time')
    pp.rcParams['figure.figsize'] = (20.0, 10.0)
    pp.show()


In [ ]:
# young data
#ydt1[ydt1[TYPE] == 'a'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
#ydt1[ydt1[TYPE] == 'b'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
#ydt1[ydt1[TYPE] == 'c'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
#ydt1[ydt1[TYPE] == 'p'].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()

for t in ydt1[TYPE].unique():
    ydt1[ydt1[TYPE] == t].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
    pp.title('Young Type ' + t)
    pp.ylabel('Target Probability')
    pp.xlabel('Time')
    pp.rcParams['figure.figsize'] = (20.0, 10.0)
    pp.show()

convert to pivot table to populate missing data cells

**NOTE**: this is a long process

In [ ]:
%%time
KEEP_COLUMNS2 = [LOAD, DIGITS, TARGET, COMP, FILL1, FILL2, RESPONSE, 'TOUCH_FIXED']

#odt121 = odt1[odt1[TIME] <= 5000]
#odt121 = odt121[odt121[TYPE] != 'p']
odt121 = odt1
ydt121 = ydt1


#ref : https://stackoverflow.com/questions/54527175/pandas-in-pivot-table-how-to-custom-fill-missing-values#54527289
odt122 = odt121.pivot_table(index=[PART, TRIAL, TYPE],
                            #values=['TOUCH_FIXED', TARGET, COMP, FILL1, FILL2],
                            values=[LOAD, DIGITS, TARGET, COMP, FILL1, FILL2, RESPONSE, 'TOUCH_FIXED'],
                            columns=[TIME],
                            aggfunc=lambda x:x,
                            fill_value=None).fillna(method='ffill', axis=1)


ydt122 = ydt121.pivot_table(index=[PART, TRIAL, TYPE],
                            #values=['TOUCH_FIXED', TARGET, COMP, FILL1, FILL2],
                            values=[LOAD, DIGITS, TARGET, COMP, FILL1, FILL2, RESPONSE, 'TOUCH_FIXED'],
                            columns=[TIME],
                            aggfunc=lambda x:x,
                            fill_value=None).fillna(method='ffill', axis=1)
odt122, ydt122

manipulate some of the data to bring it back to the original form

In [ ]:
%%time


# fix back from pivoted data
odt123 = odt122.stack().astype({'TOUCH_FIXED': int, LOAD: int})
odt124 = odt123.reset_index()

ydt123 = ydt122.stack().astype({'TOUCH_FIXED': int, LOAD: int})
ydt124 = ydt123.reset_index()

odt124, ydt124

chop on time threshold and omit practice data

In [ ]:
%%time

odt4 = odt124[odt124[TIME] <= 5000] 
odt4 = odt4[odt4[TYPE] != 'p']

ydt4 = ydt124[ydt124[TIME] <= 5000] 
ydt4 = ydt4[ydt4[TYPE] != 'p']

odt4, ydt4

cleanup: clean the practice type and clean the extra data generated when applying missing time bins earlier

In [ ]:
%%time

#ref: df[df['column name'].map(lambda x: str(x)!=".")]
# from : https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression

ydt41=ydt4[ydt4[TYPE] != 'p']
ydt42=ydt41[ydt41[LOAD].map(lambda x: x != -1)]

odt41=odt4[odt4[TYPE] != 'p']
odt42=odt41[odt41[LOAD].map(lambda x: x != -1)]


odt42,ydt42
odt4=odt42
ydt4=ydt42

plotting the graphs

In [ ]:
for t in odt4[TYPE].unique():
    odt4[odt4[TYPE] == t].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
    pp.title('Old Type ' + t)
    pp.ylabel('Target Probability')
    pp.xlabel('Time')
    pp.rcParams['figure.figsize'] = (20.0, 10.0)
    pp.show()

In [ ]:
for t in ydt4[TYPE].unique():
    ydt4[ydt4[TYPE] == t].pivot_table(index=[TIME], values=[TARGET], aggfunc=np.average).plot()
    pp.title('Young Type ' + t)
    pp.ylabel('Target Probability')
    pp.xlabel('Time')
    pp.rcParams['figure.figsize'] = (20.0, 10.0)

In [ ]:
odt4, ydt4

now saves the new data

In [ ]:
odt4.reset_index().to_csv(output_name(F_OLD_DATA, 'padded'), index=False)
ydt4.reset_index().to_csv(output_name(F_YOUNG_DATA, 'padded'), index=False)

In [ ]:
##odt5 = odt4.sort_values([PART, TRIAL, TIME])
##odt5


# more request
## new file
each part average of each type and load

column order:
* group (young, old)
* part
* time
* type
* load
----
* target average
* target - all average
* target - comp average


In [ ]:
%%time
oda = odt4.pivot_table(index=[TIME, PART, TYPE, LOAD],
                      values=[TARGET, COMP, FILL1, FILL2],
                      aggfunc=np.average)

yda = ydt4.pivot_table(index=[TIME, PART, TYPE, LOAD],
                      values=[TARGET, COMP, FILL1, FILL2],
                      aggfunc=np.average)

oda, yda


In [ ]:
# sample flow for aggregate average over PART, TIME, TIME and LOAD 
z=odt4[odt4[PART]==201]
z1=z[z[TYPE] == 'a']
z1
z2=z1[z1[TIME] == 0]
z2=z2[z2[LOAD] == 1]
z3 = z2.pivot_table(index=[PART, TIME, TYPE, LOAD],
                   values=[TARGET, COMP, FILL1, FILL2],
                   aggfunc=np.average)
z3

In [ ]:
%%time

# this is long process flow

# run a look for all the batches
# both for old and for young

ototout = pd.DataFrame()
for p in odt4[PART].unique():
    #print(f'participant {p}')
    dp=odt4[odt4[PART] == p]
    for l in [1, 4]:
        dl = dp[dp[LOAD] == l]
        for t in dl[TIME].unique():
            dt = dl[dl[TIME] == t]
            dtout = dt.pivot_table(index=[PART, TIME, TYPE, LOAD],
                                  values=[TARGET, COMP, FILL1, FILL2],
                                  aggfunc=np.average)
            ototout = pd.concat([ototout, dtout], axis=0)
            
ytotout = pd.DataFrame()
for p in ydt4[PART].unique():
    #print(f'participant {p}')
    dp=ydt4[ydt4[PART] == p]
    for l in [1, 4]:
        dl = dp[dp[LOAD] == l]
        for t in dl[TIME].unique():
            dt = dl[dl[TIME] == t]
            dtout = dt.pivot_table(index=[PART, TIME, TYPE, LOAD],
                                  values=[TARGET, COMP, FILL1, FILL2],
                                  aggfunc=np.average)
            ytotout = pd.concat([ytotout, dtout], axis=0)

In [ ]:
#fixes and sorting
oda2 = ototout.reset_index().sort_values([TYPE, LOAD, PART])
yda2 = ytotout.reset_index().sort_values([TYPE, LOAD, PART])
# concatentate 
out = pd.concat([yda2, oda2], axis=0, keys=['young', 'old'], names=['group'])
# save the average file
out.reset_index(['group']).reset_index(drop=True).to_csv('data/young_old_data_avg_custom.csv', index=False)


In [ ]:
oda1 = oda.reset_index().set_index([PART, TIME, TYPE]).sort_values([TYPE])
yda1 = yda.reset_index().set_index([PART, TIME, TYPE]).sort_values([TYPE])
oda1, yda1
out = pd.concat([yda1, oda1], axis=0, keys=['young', 'old'], names=['group'])
out

#out.reset_index().to_csv(output_name('data/young_old.csv', 'avg'), index=False)

save the avgerage as csv

In [ ]:
oda.reset_index().to_csv(output_name(F_OLD_DATA, 'avg'), index=False)
yda.reset_index().to_csv(output_name(F_YOUNG_DATA, 'avg'), index=False)


In [ ]:
oda.reset_index().pivot_table(index=[TIME, LOAD], values=[TARGET, COMP, FILL1, FILL2]).plot()
pp.show()
yda.reset_index().pivot_table(index=[TIME, LOAD], values=[TARGET, COMP, FILL1, FILL2]).plot()
pp.show()

In [ ]:
oda.reset_index().pivot_table(index=[TIME], columns=[LOAD], values=[TARGET, COMP, FILL1, FILL2]).plot()
pp.title('Old Load Average')
pp.ylabel('Probability')
pp.xlabel('Time')
pp.rcParams['figure.figsize'] = (20.0, 10.0)
pp.show()

yda.reset_index().pivot_table(index=[TIME], columns=[LOAD], values=[TARGET, COMP, FILL1, FILL2]).plot()
pp.title('Young Type ' + t)
pp.ylabel('Probability')
pp.xlabel('Time')
pp.rcParams['figure.figsize'] = (20.0, 10.0)
pp.show()